### Debugging Outputs

1.<br>
(a) length of schedule: 5940 for 30K, 5980 for 10K. The function evaluation of your solution usually falls in the range of 3000-5000<br>

2.<br>
(c) Your cost function should be a number between 2200-2400 <br>
(d) Final tempreture: 3.36K <br>
(e) You should be able to assign rank/cultivar of a wine according to centers. Using the centers you found in (b) to reassign ranks, then the cost function value would be 2160.17

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Visualize a 2d function

In [2]:
def plot_surface(func,x_min=-2,x_max=2,y_min=-2,y_max=2):
    a=np.linspace(x_min,x_max,100)
    b=np.linspace(y_min,y_max,100)
    x,y=np.meshgrid(a,b)
    z=func((x,y))
    fig=plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(x,y,z)

### Simulated annealing code Q1

Summation from 1 to 10
$𝑓(𝑥 ,𝑥 ...𝑥 )=(418.9829 * 10)−∑𝑥 *(sin(√𝑥))$

In [3]:
def schwefel(pt_arr):
    return ((418.9829 * 10) - sum(pt_arr[:] * np.sin(np.sqrt(np.abs(pt_arr[:])))))
schwefel(np.array([20,40,20]))

4227.025783417527

In [4]:
# Metropolis func
def metro_accept(temps,new_eval, cur_eval):
    boltz =  1 / (temps)
    return (np.random.random_sample() <= np.exp(-1 * boltz * (new_eval - cur_eval)))

In [5]:
def SA(solution,evaluation,delta,boundary,cooling_schedule):
    """ Simulated Annealing for minimization
    solution: np.array. Initial guess of solution
    evaluation: func. Function to evaluate solution
    delta: float. Magnitude of random displacement
    boundary: array of int/float. [lowerbound,upperbound]
    cooling_schedule: np.array. An array of temperatures for simulated annealing
    """
    best_solution = solution.copy() 
    lowest_eval = evaluation(best_solution)
    for idx,temp in enumerate(cooling_schedule):
        if idx%500 == 0:
            print("%d/%d   temp:%f"%(idx,len(cooling_schedule),temp))
        for n in range(len(solution)):
            trial = solution.copy()
            trial[n] += delta*(2*np.random.random()-1)
            if trial[n] >= boundary[0] and trial[n] <= boundary[1]:
                #fill in acceptance criterion
                if (metro_accept(temp,evaluation(trial), evaluation(solution))):
                    solution = trial
                    if evaluation(solution) < lowest_eval:
                        #update solution here
                        best_solution = solution
                        lowest_eval = evaluation(solution)
    return {"solution":best_solution, "evaluation":lowest_eval}

In [6]:
bounds = np.array([-500, 500])
sols = np.zeros(10)
cool_sched30 = np.linspace(3000, 30, 5940)
cool_sched10 = np.linspace(3000,10, 5980)

In [7]:
print("initial point: " + str(sols))
SA(sols, schwefel, 0.5, bounds, cool_sched10)

initial point: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0/5980   temp:3000.000000
500/5980   temp:2749.958187
1000/5980   temp:2499.916374
1500/5980   temp:2249.874561
2000/5980   temp:1999.832748
2500/5980   temp:1749.790935
3000/5980   temp:1499.749122
3500/5980   temp:1249.707309
4000/5980   temp:999.665496
4500/5980   temp:749.623683
5000/5980   temp:499.581870
5500/5980   temp:249.540057


{'solution': array([-16.73388164,  48.04183982, -14.60919018,   3.7851917 ,
          2.77684075, -11.76433659,   0.4707524 , -22.21454719,
         -5.81164549,  -6.18329104]),
 'evaluation': 4113.535087122041}

Linear cooling with end temp of 10 Kelvin results in final function evaluations of 4080, 4099, and 4132 and an average function evaluation of 4104. Linear cooling with end temp of 30 Kelvin results in final function evaluations of 4123, 4116, 4104 and an average function evaluation of 4114. Clearly, linear cooling with the lower end temp of 10 Kelvin resulted in a lower average final function evaluation (4104 < 4114) so I do find better solutions when cooling to the lower temperature.  

## 1b:

In [8]:
def log_cool(init_TSA):
    TSA_temps = []
    k = 0
    sig = 1000
    while k < 6001:
        TSA = init_TSA / (1 + ((init_TSA * np.log(1+k)) / (3 * sig)))
        k +=1
        TSA_temps.append(TSA)
    return np.array(TSA_temps)
atemp = log_cool(6000)[:-1]
atemp[-1] 

326.10415680714726

In [9]:
# run with same initial condition 3 times
print("initial point: " + str(sols))
SA(sols, schwefel, 0.5, bounds, atemp)

initial point: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0/6000   temp:6000.000000
500/6000   temp:446.654152
1000/6000   temp:404.926346
1500/6000   temp:383.931847
2000/6000   temp:370.306260
2500/6000   temp:360.384346
3000/6000   temp:352.663160
3500/6000   temp:346.388168
4000/6000   temp:341.130065
4500/6000   temp:336.622703
5000/6000   temp:332.690384
5500/6000   temp:329.211417


{'solution': array([-18.64681894, -20.58933935, -18.00668499, -22.62494205,
         10.49872312,   1.3339844 , -25.74080733, -28.76201993,
        -16.84036832,  12.35244367]),
 'evaluation': 4057.203737591527}

Logarithmic cooling with start temp of 3000K resulted in final function evaluations of 4050, 4136, and 4137, so it has an average function evaluation of 4107. Logarithmic cooling with start temp of 6000K resulted in final function evaluations of 4114, 4159, and 4125, so it has an average function evaluation of 4133. Thus, logarithmic cooling with a smaller start temp appears to be performing better even though both have the same number of steps. Logarithmic cooling with the start temp of 6000K is worse compared to linear cooling with start temps of either 3000K and 6000K evident from logarithmic cooling with the start temp of 6000K having the largest average function evaluation out of all of these options. Logarithmic cooling with the start temp of 3000K performed better than linear cooling with start temp of 6000K, but lost barely to linear cooling with start temp of 3000K since it had a slightly larger average function evaluation. 

## 1c: Unique Annealing 

In [10]:
def my_cool(init_TSA, alpha):
    TSA_temps = []
    k = 0
    while k < 6001:
        TSA = init_TSA * (alpha)**k 
        k +=1
        TSA_temps.append(TSA)
    return np.array(TSA_temps)
atemp = my_cool(3000, 0.9987)[:-1]
atemp[-5:-1] 

array([1.23096375, 1.2293635 , 1.22776532, 1.22616923])

In [11]:
print("initial point: " + str(sols))
SA(sols, schwefel, 0.5, bounds, atemp)

initial point: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0/6000   temp:3000.000000
500/6000   temp:1565.475203
1000/6000   temp:816.904204
1500/6000   temp:426.281092
2000/6000   temp:222.444159
2500/6000   temp:116.076939
3000/6000   temp:60.571856
3500/6000   temp:31.607913
4000/6000   temp:16.493801
4500/6000   temp:8.606879
5000/6000   temp:4.491285
5500/6000   temp:2.343665


{'solution': array([  4.72996614,   7.0170701 , -26.37967774,   5.50016291,
        -24.89777163, -27.15769714,   6.82558451, -29.78463429,
        -23.69587672,   5.1434009 ]),
 'evaluation': 4054.1560053713765}

I made my own cooling schedule starting with 3000K, 6000 steps, and ends at 1.23K. The new temps are determined via geometric cooling schedule with the multiplier being 0.9987. This schedule produced final function evaluations of 4015, 4057, and 4011, so it had an average function evaluation of 4028. This final average function evaluation was better than final average function evaluation produced by the cooling schedules used in 1a and 1b, so I was able to find a better solution. 

In [12]:
ex_sols = np.array([-25.23234948,   7.5678464 , -22.53009957,   6.80814302,
        -25.40759713,   1.93960011, -24.05591988, -27.7639376 ,
          5.53715501,   4.23131456])

In [13]:
# Code from Tutorial #1
def golden_section(func, start, end, reference, tol):
    if (end - start).sum() < (tol * 10):
        return {'x':reference, 'y':func(reference)}
    else:
        if np.abs(reference - start).sum() < np.abs(end - reference).sum():
            new_reference = end - (end - reference)*0.618
            if func(new_reference) > func(reference):
                return golden_section(func, start, new_reference, reference, tol)
            else:
                return golden_section(func, reference, end, new_reference, tol)
        else:
            new_reference = start + (reference-start)* 0.618
            if func(new_reference) > func(reference):
                return golden_section(func, new_reference, end, reference, tol)
            else:
                return golden_section(func, start, reference, new_reference, tol)
            
golden_section(schwefel,ex_sols - 10, sols + 10, np.ones(10) * (20 *0.618), 1e-5)

{'x': array([-24.84492851,   0.79635536, -22.73246682,   0.20246348,
        -24.98192692,  -3.60347998, -23.92526432, -26.82397674,
         -0.79112097,  -1.81195102]),
 'y': 4076.8193753917394}

One of my classical simulated annealing results was a solution: [-25.23234948,   7.5678464 , -22.53009957,   6.80814302, -25.40759713,   1.93960011, -24.05591988, -27.7639376 ,
5.53715501,   4.23131456] and had a final function evaluation of 4056. I tried golden section with an x-value search range with (my solution - 10 (substraction from each component)) as the start point and an end point of a vector with components of the solution + 10. I consistently ended up with a final function evaluation of 4076, which is the larger than the result outputted from CSA, so I could not find an even better solution via golden section on the CSA solution. 

## 2a:

In [14]:
df_wines = pd.read_csv("wines.csv")
df_wines.head()

,Alcohol %,Malic Acid,Ash,Alkalinity,Mg,Phenols,Flavanoids,Phenols.1,Proantho-cyanins,Color intensity,Hue,OD280 315,Proline,Start assignment,ranking
0,14.23,1.71,2.43,15.6,127,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065,1,1
1,13.24,2.59,2.87,21.0,118,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735,1,1
2,14.83,1.64,2.17,14.0,97,2.8,2.98,0.29,1.98,5.20,1.08,2.85,1045,1,1
3,14.12,1.48,2.32,16.8,95,2.2,2.43,0.26,1.57,5.00,1.17,2.82,1280,1,1
4,13.75,1.73,2.41,16.0,89,2.6,2.76,0.29,1.81,5.60,1.15,2.90,1320,1,1


In [15]:
df_futwines = df_wines.copy()
df_futwines.drop(columns = ['Start assignment', 'ranking'], inplace = True)
df_futwines.head()

,Alcohol %,Malic Acid,Ash,Alkalinity,Mg,Phenols,Flavanoids,Phenols.1,Proantho-cyanins,Color intensity,Hue,OD280 315,Proline
0,14.23,1.71,2.43,15.6,127,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.24,2.59,2.87,21.0,118,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735
2,14.83,1.64,2.17,14.0,97,2.8,2.98,0.29,1.98,5.20,1.08,2.85,1045
3,14.12,1.48,2.32,16.8,95,2.2,2.43,0.26,1.57,5.00,1.17,2.82,1280
4,13.75,1.73,2.41,16.0,89,2.6,2.76,0.29,1.81,5.60,1.15,2.90,1320


In [16]:
df_nrml = (df_futwines - df_futwines.mean()) / np.std(df_futwines)
df_nrml.head()

,Alcohol %,Malic Acid,Ash,Alkalinity,Mg,Phenols,Flavanoids,Phenols.1,Proantho-cyanins,Color intensity,Hue,OD280 315,Proline
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874
2,2.259772,-0.625086,-0.718336,-1.650049,-0.192495,0.808997,0.954502,-0.578985,0.681738,0.061386,0.537671,0.336606,0.949319
3,1.382733,-0.768712,-0.170035,-0.809251,-0.332922,-0.152402,0.402320,-0.820719,-0.036617,-0.025128,0.932531,0.294232,1.697675
4,0.925685,-0.544297,0.158946,-1.049479,-0.754202,0.488531,0.733629,-0.578985,0.383884,0.234414,0.844785,0.407228,1.825055


## 2b:

In [17]:
df_nrml['Start assignment'] = df_wines['Start assignment']
df_centroids = df_nrml.groupby('Start assignment').agg(np.mean)
df_centroids

,Alcohol %,Malic Acid,Ash,Alkalinity,Mg,Phenols,Flavanoids,Phenols.1,Proantho-cyanins,Color intensity,Hue,OD280 315,Proline
Start assignment,,,,,,,,,,,,,
1,-0.026321,-0.022878,0.039202,-0.011425,0.001197,0.046232,-0.014499,-0.092738,0.015342,-0.122680,0.072159,-0.021132,-0.001088
2,-0.030284,-0.043279,-0.117993,-0.122667,-0.180594,-0.110306,-0.040446,0.035593,-0.147087,-0.218465,0.084808,0.077818,-0.085479
3,0.054317,0.063613,0.076850,0.129509,0.173535,0.062731,0.052907,0.053751,0.127677,0.327948,-0.150638,-0.055173,0.083711


## 2c:

In [18]:
def cost_func_eval(df_nml, df_cent):
    cost_part = 0
    for i in range(1,4):
        temp = df_nml[df_nml['Start assignment'] == i]
        cost_part +=((temp - df_cent.iloc[i-1])**2).sum().sum()
    return cost_part
cost_func_eval(df_nrml, df_centroids)

2288.2043588628485

## 2d:

In [19]:
rank_asn = df_nrml['Start assignment'].values
cents = df_centroids.values
len(rank_asn)

178

In [20]:
import numba
@numba.njit()
def calc_cost(df, centers, current_rank):
    """
    df: np.array of defined type, wine features
    current_rank: np.array of defined type (178,), wine assignment
    centers: np.array of defined type (3, 13), centroids
    
    numba will throw errors on pd.DataFrame/Series with undefined types
    use DataFrame.values to get np.array object instead of pd.DataFrame
    """
    total_dist=0
    df[:,13] = current_rank
    for r in [1,2,3]:
        # find wines with r rank, calculate & add to the total cost
        temp = df[df[:,13] == r][:,:13]
        size_mat = temp.shape
        ac_size = size_mat[0] * size_mat[1]
        total_dist += np.sum(((temp - centers[r-1])**2).reshape(1,ac_size))
    return total_dist
calc_cost(df_nrml.values, cents, rank_asn)

2288.2043588628485

In [21]:
def simulated_annealing(feats,ranks,centers,start_temp,alpha,steps=10000):
    """ Simulated Annealing for clustering
    feats: pd.DataFrame. Normalized chemical descriptors
    ranks: np.array shape(178,). Initial assignment.
    centers: np.array shape (3,13). Fixed centers
    start_temp: float. Initial temperature
    alpha: float. Hyperparameter for geometric cooling
    steps: int. 
    """
    best_rank=ranks.copy()
    # evaluate the cost function with current best rank
    lowest_eval= calc_cost(feats.values,centers,best_rank)
    for step in (range(steps)):
        # update tempture according to geometric cooling schedule
        temp = start_temp * ((alpha)**(step+1))
        if step%500==0: # original -> if step%500 == 0:
            print(step,temp,lowest_eval)
        for n in range(len(ranks)):
            trial=ranks.copy()
            rand_choice=np.random.randint(3)+1
            trial[n]=rand_choice
            # Metropolis acceptance criterion
            if (metro_accept(temp,calc_cost(feats.values, centers, trial), calc_cost(feats.values, centers, ranks))):
                ranks=trial
                # update evaluation
                new_eval= calc_cost(feats.values,centers,ranks)
                if new_eval<lowest_eval:
                    #update best rank and lowest_eval
                    best_rank = ranks.copy()
                    lowest_eval= calc_cost(feats.values,centers,best_rank)
    return {"solution":best_rank,"evaluation":lowest_eval}

To validate how good is your solution

In [22]:
def validate(solution, df):
    """Prints out how many wines are corretly assigned to its cultivar
    solution: np.array shape(178,). Your solution.
    df: pd.DataFrame. Read-in of the wines.csv dataset
    """
    # correct classification
    ranking = df['ranking'].values
    cluster_1 = list(df[df['ranking']==1].index)
    cluster_2 = list(df[df['ranking']==2].index)
    cluster_3 = list(df[df['ranking']==3].index)
    clusters =[cluster_1,cluster_2,cluster_3]

    for i in range(3):
        scores=[]
        for j in range(1,4):
            sol_j= [idx for idx,k in enumerate(solution) if k==j]
            scores.append(len(np.intersect1d(sol_j, clusters[i])))

        print(f'Class {i+1} - cultivar {np.argmax(scores)+1}: {np.max(scores)} out \
of {len(clusters[np.argmax(scores)])} are classified correctly')

In [23]:
prayge = simulated_annealing(df_nrml, rank_asn, cents, 500, 0.999, 5000)

0 499.5 2288.2043588628485
500 302.88628295816176 2285.668123782146
1000 183.66386467309636 2274.272927459844
1500 111.3699004695996 2274.272927459844
2000 67.53236273605097 2274.272927459844
2500 40.95020285986938 2274.272927459844
3000 24.831340802019064 2274.272927459844
3500 15.057202235016803 2274.272927459844
4000 9.130370403830963 2274.272927459844
4500 5.536464371666808 2268.493919179643


In [24]:
prayge

{'solution': array([1, 1, 1, 1, 1, 3, 1, 3, 2, 2, 3, 2, 3, 2, 3, 2, 3, 3, 3, 3, 2, 2,
        2, 2, 3, 2, 3, 2, 1, 2, 3, 2, 1, 1, 1, 2, 1, 3, 2, 2, 2, 2, 1, 3,
        1, 2, 2, 2, 2, 2, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 1, 1, 2, 3, 1, 3,
        3, 1, 3, 3, 1, 2, 2, 3, 1, 3, 3, 1, 2, 2, 3, 2, 1, 3, 2, 2, 2, 2,
        3, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 3, 2, 2, 3, 2, 3, 3,
        3, 3, 2, 2, 1, 1, 1, 2, 3, 3, 1, 2, 1, 1, 3, 2, 1, 3, 2, 3, 1, 1,
        3, 1, 3, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 3, 1, 2, 2, 1, 2, 1, 2, 2,
        3, 1, 3, 2, 3, 3, 1, 2, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 2, 3, 3,
        3, 2]),
 'evaluation': 2247.2820058411708}

In [25]:
validate(prayge['solution'], df_wines)

Class 1 - cultivar 3: 27 out of 48 are classified correctly
Class 2 - cultivar 2: 37 out of 71 are classified correctly
Class 3 - cultivar 3: 27 out of 48 are classified correctly


In [26]:
prayge2 = simulated_annealing(df_nrml, rank_asn, cents, 500, 0.999, 5000)

0 499.5 2288.2043588628485
500 302.88628295816176 2279.8944615637756
1000 183.66386467309636 2279.8944615637756
1500 111.3699004695996 2279.8944615637756
2000 67.53236273605097 2279.8944615637756
2500 40.95020285986938 2279.8944615637756
3000 24.831340802019064 2279.8944615637756
3500 15.057202235016803 2279.8944615637756
4000 9.130370403830963 2269.491528755392
4500 5.536464371666808 2269.274188842376


In [27]:
prayge2

{'solution': array([1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 3, 1, 1, 3, 2, 2,
        1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 2, 1, 1, 1, 2, 2,
        1, 2, 2, 2, 3, 3, 2, 2, 3, 3, 1, 3, 3, 2, 2, 1, 3, 2, 1, 3, 1, 1,
        1, 3, 3, 3, 1, 3, 2, 3, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 2, 3, 2,
        2, 2, 3, 2, 2, 2, 2, 2, 3, 3, 1, 2, 3, 1, 2, 1, 1, 2, 2, 2, 1, 3,
        3, 3, 2, 2, 1, 2, 2, 3, 1, 1, 2, 3, 3, 2, 3, 2, 2, 3, 1, 3, 3, 1,
        3, 3, 3, 3, 2, 1, 1, 2, 1, 1, 1, 2, 3, 2, 1, 1, 3, 1, 1, 2, 2, 2,
        3, 3, 2, 2, 1, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 1,
        3, 2]),
 'evaluation': 2249.0929759002834}

In [28]:
validate(prayge2['solution'], df_wines)

Class 1 - cultivar 1: 25 out of 59 are classified correctly
Class 2 - cultivar 2: 32 out of 71 are classified correctly
Class 3 - cultivar 3: 22 out of 48 are classified correctly


In [29]:
prayge3 = simulated_annealing(df_nrml, rank_asn, cents, 500, 0.999, 5000)

0 499.5 2288.2043588628485
500 302.88628295816176 2287.8530359178967
1000 183.66386467309636 2287.8530359178967
1500 111.3699004695996 2287.8530359178967
2000 67.53236273605097 2287.8530359178967
2500 40.95020285986938 2287.8530359178967
3000 24.831340802019064 2287.8530359178967
3500 15.057202235016803 2284.277312508752
4000 9.130370403830963 2273.7448148795265
4500 5.536464371666808 2262.1419621581204


In [30]:
prayge3

{'solution': array([1, 3, 2, 1, 3, 1, 3, 1, 3, 1, 1, 3, 1, 3, 3, 3, 1, 2, 3, 3, 2, 2,
        3, 2, 2, 2, 2, 2, 2, 1, 2, 3, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1,
        2, 1, 2, 2, 3, 3, 3, 3, 1, 1, 3, 3, 3, 1, 3, 1, 2, 2, 1, 1, 1, 3,
        2, 2, 2, 2, 2, 3, 2, 3, 2, 3, 3, 2, 2, 2, 1, 2, 3, 3, 2, 3, 2, 2,
        2, 2, 2, 1, 2, 2, 1, 1, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 1, 2,
        3, 3, 2, 3, 1, 3, 2, 3, 1, 3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 3, 2, 3,
        3, 1, 3, 3, 2, 3, 3, 1, 3, 3, 2, 1, 3, 1, 2, 2, 2, 2, 1, 3, 3, 2,
        3, 1, 2, 1, 2, 3, 2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 2, 1,
        3, 1]),
 'evaluation': 2250.828424161473}

In [31]:
validate(prayge3['solution'], df_wines)

Class 1 - cultivar 3: 28 out of 48 are classified correctly
Class 2 - cultivar 2: 39 out of 71 are classified correctly
Class 3 - cultivar 3: 27 out of 48 are classified correctly


From the 3 attempts, I am hovering around 50% (plus or minus 10%) of the wines in each class being correctly classified and final function evaluations in the range of 2245 to 2255. This is not a great classification, but it can definitely be improved. 

## 2e:

In [32]:
def closest_cent(df,new_cents):
    ranks_arr = []
    for i in range(178):
        dists_cent1 = ((df[i,:13] - new_cents[0])**2).sum()
        dists_cent2 = ((df[i,:13] - new_cents[1])**2).sum()
        dists_cent3 = ((df[i, :13] - new_cents[2])**2).sum()
        min_dist = np.min([dists_cent1, dists_cent2, dists_cent3])
        new_rank = np.where(np.array([dists_cent1, dists_cent2, dists_cent3]) == min_dist)
        ranks_arr.append(new_rank[0].item() + 1)
    return np.array(ranks_arr)

In [33]:
def up_simulated_annealing(feats,ranks,centers,start_temp,alpha,steps=10000):
    """ Simulated Annealing for clustering
    feats: pd.DataFrame. Normalized chemical descriptors
    ranks: np.array shape(178,). Initial assignment.
    centers: np.array shape (3,13). Fixed centers
    start_temp: float. Initial temperature
    alpha: float. Hyperparameter for geometric cooling
    steps: int. 
    """
    best_rank=ranks.copy()
    # evaluate the cost function with current best rank
    lowest_eval= calc_cost(feats.values,centers,best_rank)
    for step in (range(steps)):
        # update tempture according to geometric cooling schedule
        temp = start_temp * ((alpha)**(step+1))
        if step%500==0: # original -> if step%500 == 0:
            print(step,temp,lowest_eval)
        trial_cents = centers.copy()
        mods_cents = [0.01*(2*np.random.random()-1) for i in range(39)]
        ncents = np.ones((3,13))
        ncents[0] = trial_cents[0] + mods_cents[:13]
        ncents[1] = trial_cents[1] + mods_cents[13:26]
        ncents[2] = trial_cents[2] + mods_cents[26:]
        new_rnks = closest_cent(feats.values, ncents)
        # Metropolis acceptance criterion
        if (metro_accept(temp,calc_cost(feats.values, centers, new_rnks), calc_cost(feats.values, centers, ranks))):
            ranks=new_rnks
            # update evaluation
            new_eval= calc_cost(feats.values,centers,ranks)
            if new_eval<lowest_eval:
                #update best rank and lowest_eval
                best_rank = ranks.copy()
                lowest_eval= calc_cost(feats.values,centers,best_rank)
    return {"solution":best_rank,"evaluation":lowest_eval}

In [34]:
CSA_rng_cents = up_simulated_annealing(df_nrml, rank_asn, cents, 500, 0.999, 5000)

0 499.5 2288.2043588628485
500 302.88628295816176 2160.1710142099078
1000 183.66386467309636 2160.1710142099078
1500 111.3699004695996 2160.1710142099078
2000 67.53236273605097 2160.1710142099078
2500 40.95020285986938 2160.1710142099078
3000 24.831340802019064 2160.1710142099078
3500 15.057202235016803 2160.1710142099078
4000 9.130370403830963 2160.1710142099078
4500 5.536464371666808 2160.1710142099078


In [35]:
CSA_rng_cents

{'solution': array([3, 3, 1, 1, 1, 3, 3, 1, 1, 2, 1, 3, 1, 1, 1, 1, 3, 1, 3, 3, 2, 1,
        2, 2, 2, 2, 1, 2, 2, 2, 3, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 3,
        1, 2, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 3, 1, 3,
        1, 3, 1, 1, 2, 3, 1, 3, 3, 1, 3, 3, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3,
        3, 3, 3, 2, 3, 2, 3, 3, 1, 3, 3, 3, 3, 3, 1, 1, 1, 3, 1, 3, 2, 3,
        3, 3, 3, 3, 2, 1, 2, 1, 3, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2,
        3, 1, 1, 2, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 2]),
 'evaluation': 2160.1710142099078}

In [36]:
validate(CSA_rng_cents['solution'], df_wines)

Class 1 - cultivar 3: 32 out of 48 are classified correctly
Class 2 - cultivar 2: 51 out of 71 are classified correctly
Class 3 - cultivar 3: 33 out of 48 are classified correctly


In [37]:
CSA_rng_cents2 = up_simulated_annealing(df_nrml, rank_asn, cents, 500, 0.999, 5000)

0 499.5 2288.2043588628485
500 302.88628295816176 2160.1710142099078
1000 183.66386467309636 2160.1710142099078
1500 111.3699004695996 2160.1710142099078
2000 67.53236273605097 2160.1710142099078
2500 40.95020285986938 2160.1710142099078
3000 24.831340802019064 2160.1710142099078
3500 15.057202235016803 2160.1710142099078
4000 9.130370403830963 2160.1710142099078
4500 5.536464371666808 2160.1710142099078


In [38]:
validate(CSA_rng_cents2['solution'], df_wines)

Class 1 - cultivar 3: 32 out of 48 are classified correctly
Class 2 - cultivar 2: 51 out of 71 are classified correctly
Class 3 - cultivar 3: 33 out of 48 are classified correctly


I ran this variation of CSA 3 times and I consistently ended with a final function value of 2160.171. This is a better solution than the version of CSA used in part 2d and I have never seen a final function evaluation in 2d smaller than the final function evaluation in 2e. Overall, for each class, on average, around 2/3 or 67% of the wines in each class are correctly classified.  For class 1, 32 out of 48 were classified correctly. For class 2, 51 out of 71 were classified correctly. For class 3, 33 out of 48 were classified correctly. 